![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Visualizing Data with Matplotlib
#### Line Plots, Histograms, Scatter plots, and Heat Maps

### Import libraries
Let's start by importing the libraries we need.

In [1]:
import matplotlib.pyplot as plt
import numpy as np  # used later for OLS

### Visualizing SPY
We will visualize SPY using a time-series line plot and a histogram.

#### Gathering Data
To begin, we retrieve daily close data for SPY

In [2]:
qb = QuantBook()
spy = qb.AddEquity("SPY")
history = qb.History(qb.Securities.Keys, 360, Resolution.Daily)
spy_hist = history.loc['SPY']['close']
spy_hist

Here, we visualize SPY with a time-series line plot. Note that pandas has built-in support for matplotlib, making it easier and quicker for us to create plots.

In [4]:
fig, ax = plt.subplots()
spy_hist.plot(ax=ax) 
plt.title('SPY Daily Close')

Next, we want to visualize the returns distribution of SPY. We call pct_change() on our SPY Series to get the daily returns.

In [5]:
spy_daily_ret = spy_hist.pct_change()
plt.hist(spy_daily_ret, bins=50)
plt.title('Are normally distributed returns a hoax?')
plt.xlabel('Daily Return')
plt.ylabel('Count')

### Visualizing the Correlation Between GLD and BAR Returns
GLD and BAR are two different Gold-tracking ETFs, and we would like to visualize their correlation. To do this, we will employ a scatter plot of their returns.

#### Gathering Data
First, we get daily close returns data for GLD and BAR

In [6]:
qb.AddEquity('GLD')
qb.AddEquity('BAR') 

gold_etf_hist = qb.History(['GLD', 'BAR'], 300, Resolution.Daily)

gld_ret = gold_etf_hist.loc['GLD']['close'].pct_change().dropna()
bar_ret = gold_etf_hist.loc['BAR']['close'].pct_change().dropna()

Since we would like to include a trend line (OLS) in our scatter plot, we will need to employ numpy to calculate our OLS line.

In [12]:
# get coefficients of the OLS Line (trend line)
m, b = np.polyfit(gld_ret, bar_ret, deg=1)

x = np.linspace(-.045, .06)
plt.plot(x, x * m + b, color='red')

plt.scatter(gld_ret, bar_ret)

plt.title('GLD vs BAR daily returns Scatter Plot')
plt.xlabel('GLD')
plt.ylabel('BAR')
plt.show()

### Visualizing Correlations Between Banking Stocks
Correlations between stocks in the same sector are commonly applied to develop pairs trading strategies. We will visualize the correlations between a few stocks in the banking sector using a heat map. Note: we strongly advise using Seaborn instead for heat maps, as it needs only roughly 1/10th of the number of lines of code.

#### Gathering Data
First, we get historical closing prices for four banking sector stocks.

In [23]:
tickers = [
    "COF", # Capital One Financial Corporation
    "JPM", # J P Morgan Chase & Co
    "STI", # SunTrust Banks, Inc.
    "WFC" # Wells Fargo & Company
]
symbols = [qb.AddEquity(ticker, Resolution.Daily).Symbol for ticker in tickers]

history = qb.History(tickers, 
                     datetime(2020, 2, 1), 
                     datetime(2020, 7, 1), 
                     Resolution.Daily)

df = history['close'].unstack(level=0)

Before we create the heat map, we must create a correlation matrix, which has all the correlations between all possible pairs of symbols. This can be easily done with pandas DataFrame's built-in corr() method.

In [26]:
corr_matrix = df.corr()
corr_matrix

With the correlation matrix, we are ready to create a heat map. 

In [37]:
fig, ax = plt.subplots()
im = ax.imshow(corr_matrix)

# full symbol data will clutter the heat map
tickers_ordered = [symbol.split()[0] for symbol in corr_matrix.columns]

# matplotlib has a bug where the first tick doesn't appear
# so we've put in a place holder
# we STRONGLY advise using Seaborn for heat maps
ax.set_xticklabels(['-'] + tickers_ordered)
ax.set_yticklabels(['-'] + tickers_ordered)

# make sure x and y-axes don't have too many ticks
ax.locator_params(axis='x', nbins=len(tickers_ordered))
ax.locator_params(axis='y', nbins=len(tickers_ordered))

# Rotate the x tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(corr_matrix.index)):
    for c in range(len(corr_matrix.columns)):
        label = round(corr_matrix.iloc[i][corr_matrix.columns[c]], 2)
        text = ax.text(c, i, label,
                       ha="center", va="center", color="w")

ax.set_title("Banking Stocks Correlation Heat Map")